# Reviewer 2, Question 6:

Therapeutics: the authors showed that one-third of patients required additional medication beyond beta-blockers. Since the effectiveness of flecainide in CPVT patients has been established, the combination of the beta-blocker and flecainide would be the first-line therapy. If authors limit their analysis to studies conducted after the use of flecainide, the incidence of the combination therapy will likely be higher.

HRS/EHRA/APHRS Expert Consensus Statement on the Diagnosis and Management of Patients with Inherited Primary Arrhythmia Syndromes: Document endorsed by HRS, EHRA, and APHRS in May 2013 and by ACCF, AHA, PACES, and AEPC in June 2013. https://doi.org/10.1016/j.hrthm.2013.05.014

In [50]:
import pandas as pd
import numpy as np
from pathlib import Path

In [51]:
from pathlib import Path
from analysis.database import get_config

config = get_config()
data_all = pd.read_excel(
    Path.cwd().parent / "data" / f"individual_all_data-{config.version}.xlsx")
data_all.head()

,Unnamed: 0,individual_id,sex,publication_id,title,first_author,reference,doi,year,publication_database,...,treatment_taken::Implantable cardioverter-defibrillator,treatment_taken::Left cardiac sympathetic denervation,treatment_taken::Verapamil,effective::Beta blocker,effective::Catheter ablation,effective::Enalapril,effective::Flecainide,effective::Implantable cardioverter-defibrillator,effective::Left cardiac sympathetic denervation,effective::Verapamil
0,0,1,NaN,1,Familial Evaluation in Catecholaminergic Polym...,Van Der Werf C,NaN,10.1161/CIRCEP.112.970517,2012,PubMed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,female,2,Flecainide therapy reduces exercise-induced ve...,"van der Werf, C",NaN,10.1016/j.jacc.2011.01.026,2011,PubMed,...,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN
2,2,3,male,3,Structural abnormalities on cardiac magnetic r...,"Gerber, D",NaN,10.1016/j.jacep.2020.03.006,2020,PubMed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,female,4,Genetic Background of Catecholaminergic Polymo...,"Kawamura, M",NaN,10.1253/circj.cj-12-1460,2013,PubMed,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,male,5,Gender differences in the inheritance mode of ...,"Ohno, S.",NaN,10.1371/journal.pone.0131517,2015,PubMed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
MIN_PATIENTS = 10

In [53]:
cols = [
    "individual_id",
    "Catecholaminergic polymorphic ventricular tachycardia 1",
    "p_hgvs_string",
    "publication_id",
    "first_author",
    "doi",
    "year",
    "resource_uri",
    "treatment_taken::Beta blocker",
    "treatment_taken::Flecainide",
]

data = data_all[cols].copy()

data.drop_duplicates(subset=["individual_id"], inplace=True)
# only CPVT 
data = data[
    data["Catecholaminergic polymorphic ventricular tachycardia 1"] == True]
# only must be taking beta blockers
data = data[data["treatment_taken::Beta blocker"] == True]


def get_p_posedit(x: str | float):
    if pd.isna(x):
        return x
    return x.split(".")[-1].replace("(", "").replace(")", "")


data["p_edit_str"] = data["p_hgvs_string"].apply(
    get_p_posedit
)

print("Number of missing p_edit_str:", data["p_edit_str"].isna().sum())
data = data.dropna(subset=["p_edit_str"])

print(data.shape)
data.head()

data.head()

Number of missing p_edit_str: 9
(423, 11)


,individual_id,Catecholaminergic polymorphic ventricular tachycardia 1,p_hgvs_string,publication_id,first_author,doi,year,resource_uri,treatment_taken::Beta blocker,treatment_taken::Flecainide,p_edit_str
1,2,1.0,NP_001026.2:p.(Tyr4962Cys),2,"van der Werf, C",10.1016/j.jacc.2011.01.026,2011,https://pubmed.ncbi.nlm.nih.gov/21616285,1.0,1.0,Tyr4962Cys
3,4,1.0,NP_001026.2:p.(Tyr4725Cys),4,"Kawamura, M",10.1253/circj.cj-12-1460,2013,https://pubmed.ncbi.nlm.nih.gov/23595086,1.0,0.0,Tyr4725Cys
11,12,1.0,NP_001026.2:p.(Tyr4080Cys),12,"Ostby, S.",10.1016/j.jacep.2016.01.020,2016,https://pubmed.ncbi.nlm.nih.gov/29766881,1.0,1.0,Tyr4080Cys
26,27,1.0,NP_001026.2:p.Trp4949Arg,27,"Andrsova, I.",10.1111/j.1540-8159.2012.03399.x,2012,https://pubmed.ncbi.nlm.nih.gov/22519458,1.0,0.0,Trp4949Arg
27,28,1.0,NP_001026.2:p.(Trp4645Arg),28,"Beery, TA",10.1177/1099800409333369,2009,https://pubmed.ncbi.nlm.nih.gov/19398417,1.0,0.0,Trp4645Arg


In [54]:
import re
from Bio.SeqUtils import seq1

hgvs_aa3 = set(
    "Ala Cys Asp Glu Phe Gly His Ile Lys Leu Met Asn Pro Gln Arg Ser Thr Val Trp Tyr Asx Glx Xaa Sec".split()
) | {"Ter"}


def aa3_to_aa1(posedit_str: str):
    # remove all non A-Za-z characters with regex
    aa3_codes = re.findall("[A-Z][a-z]{2}", posedit_str)

    # Convert each 3-letter code to a 1-letter code and replace it in the string
    for aa3 in aa3_codes:
        if aa3 not in hgvs_aa3:
            raise ValueError(f"Invalid AA3 code: {aa3}")
        aa1 = seq1(aa3)
        posedit_str = posedit_str.replace(aa3, aa1)

    return posedit_str


data["p_edit_str_aa1"] = data["p_edit_str"].apply(aa3_to_aa1)
data.head()

,individual_id,Catecholaminergic polymorphic ventricular tachycardia 1,p_hgvs_string,publication_id,first_author,doi,year,resource_uri,treatment_taken::Beta blocker,treatment_taken::Flecainide,p_edit_str,p_edit_str_aa1
1,2,1.0,NP_001026.2:p.(Tyr4962Cys),2,"van der Werf, C",10.1016/j.jacc.2011.01.026,2011,https://pubmed.ncbi.nlm.nih.gov/21616285,1.0,1.0,Tyr4962Cys,Y4962C
3,4,1.0,NP_001026.2:p.(Tyr4725Cys),4,"Kawamura, M",10.1253/circj.cj-12-1460,2013,https://pubmed.ncbi.nlm.nih.gov/23595086,1.0,0.0,Tyr4725Cys,Y4725C
11,12,1.0,NP_001026.2:p.(Tyr4080Cys),12,"Ostby, S.",10.1016/j.jacep.2016.01.020,2016,https://pubmed.ncbi.nlm.nih.gov/29766881,1.0,1.0,Tyr4080Cys,Y4080C
26,27,1.0,NP_001026.2:p.Trp4949Arg,27,"Andrsova, I.",10.1111/j.1540-8159.2012.03399.x,2012,https://pubmed.ncbi.nlm.nih.gov/22519458,1.0,0.0,Trp4949Arg,W4949R
27,28,1.0,NP_001026.2:p.(Trp4645Arg),28,"Beery, TA",10.1177/1099800409333369,2009,https://pubmed.ncbi.nlm.nih.gov/19398417,1.0,0.0,Trp4645Arg,W4645R


Number of patients taking beta blockers (ALL patients)

In [55]:
df_cpvt = data[
    data["Catecholaminergic polymorphic ventricular tachycardia 1"] == 1
    ]
df_cpvt_bb = data[
    data["treatment_taken::Beta blocker"] == 1
    ].copy()

In [56]:
bb_counts_all = df_cpvt_bb.groupby(
    "p_edit_str_aa1"
).agg(
    {
        "treatment_taken::Beta blocker": "sum",
    }
    # rename to be more descriptive
).rename(
    columns={
        "treatment_taken::Beta blocker": "count_beta_blockers"
    }
).sort_values(
    by="count_beta_blockers",
    ascending=False
)

bb_counts_all.head()

,count_beta_blockers
p_edit_str_aa1,
G357S,94.0
R420Q,24.0
G2337V,21.0
R420W,17.0
M3978I,14.0


In [57]:
flec_counts_all = df_cpvt_bb.groupby(
    "p_edit_str_aa1"
).agg(
    {
        "treatment_taken::Flecainide": "sum",
    }
    # rename to be more descriptive
).rename(
    columns={
        "treatment_taken::Flecainide": "count_flecainide"
    }
).sort_values(
    by="count_flecainide",
    ascending=False
)
# flec + BB
flec_counts_all.head()

,count_flecainide
p_edit_str_aa1,
R420W,12.0
M3978I,9.0
S4124G,7.0
C2277R,6.0
V4771I,4.0


In [58]:
# only variants with more than 10 patients
variants_top_by_bb_usage = pd.DataFrame(df_cpvt_bb.groupby(
    "p_edit_str_aa1"
)["individual_id"].count().sort_values(ascending=False))

variants_top_by_bb_usage = variants_top_by_bb_usage[
    variants_top_by_bb_usage["individual_id"] > MIN_PATIENTS
    ]

variants_top_by_bb_usage.columns = ["Total"]
variants_top_by_bb_usage

,Total
p_edit_str_aa1,
G357S,94
R420Q,24
G2337V,21
R420W,17
M3978I,14
G3946S,11


In [59]:
org_fisher_flec = variants_top_by_bb_usage.join(flec_counts_all,
                                                how="left").sort_values(
    "Total", ascending=False)
org_fisher_flec = org_fisher_flec.rename({
    "count_flecainide": "Flecainide_and_beta_blocker",
}, axis=1)
org_fisher_flec["Beta_blocker_only"] = org_fisher_flec["Total"] - \
                                       org_fisher_flec[
                                           "Flecainide_and_beta_blocker"]
# reorder
org_fisher_flec = org_fisher_flec[
    ["Flecainide_and_beta_blocker", "Beta_blocker_only", "Total"]]

org_fisher_flec.index.name = "p_posedit_aa1"
org_fisher_flec

,Flecainide_and_beta_blocker,Beta_blocker_only,Total
p_posedit_aa1,,,
G357S,0.0,94.0,94
R420Q,2.0,22.0,24
G2337V,0.0,21.0,21
R420W,12.0,5.0,17
M3978I,9.0,5.0,14
G3946S,2.0,9.0,11


In [84]:
after_2013 = df_cpvt_bb[df_cpvt_bb["year"] >= 2013]
before_2013 = df_cpvt_bb[df_cpvt_bb["year"] < 2013]

bb_counts_after = after_2013.groupby(
    "p_edit_str_aa1"
).agg(
    {
        "treatment_taken::Beta blocker": "sum",
    }
    # rename to be more descriptive
).rename(
    columns={
        "treatment_taken::Beta blocker": "count_beta_blockers"
    }
).sort_values(
    by="count_beta_blockers",
    ascending=False
)
# keep only the variants from the ORIGINAL list
bb_counts_after = bb_counts_after.join(variants_top_by_bb_usage, how="inner")

bb_counts_after = bb_counts_after[["count_beta_blockers"]]

bb_counts_after

,count_beta_blockers
p_edit_str_aa1,
G357S,91.0
G2337V,21.0
R420Q,15.0
R420W,9.0
G3946S,7.0
M3978I,3.0


In [85]:
flec_counts_after = after_2013.groupby(
    "p_edit_str_aa1"
).agg(
    {
        "treatment_taken::Flecainide": "sum",
    }
    # rename to be more descriptive
).rename(
    columns={
        "treatment_taken::Flecainide": "count_flecainide"
    }
).sort_values(
    by="count_flecainide",
    ascending=False
)


In [86]:
fisher_flec_after = bb_counts_after.join(flec_counts_after,
                                         how="left").sort_values(
    "count_beta_blockers", ascending=False)
fisher_flec_after = fisher_flec_after.rename({
    "count_beta_blockers": "Total",
    "count_flecainide": "Flecainide_and_beta_blocker",
}, axis=1)

fisher_flec_after["Beta_blocker_only"] = fisher_flec_after["Total"] - \
                                         fisher_flec_after[
                                             "Flecainide_and_beta_blocker"]
# reorder
fisher_flec_after = fisher_flec_after[
    ["Flecainide_and_beta_blocker", "Beta_blocker_only", "Total"]]
# name index
fisher_flec_after.index.name = "p_posedit_aa1"
fisher_flec_after

,Flecainide_and_beta_blocker,Beta_blocker_only,Total
p_posedit_aa1,,,
G357S,0.0,91.0,91.0
G2337V,0.0,21.0,21.0
R420Q,1.0,14.0,15.0
R420W,7.0,2.0,9.0
G3946S,1.0,6.0,7.0
M3978I,1.0,2.0,3.0


In [87]:
flec_counts_after.head()

,count_flecainide
p_edit_str_aa1,
R420W,7.0
C2277R,6.0
I4587V,3.0
D3638A,3.0
S4124G,3.0


In [88]:
# export to csv
org_fisher_flec.to_csv(
    Path.cwd().parent / "data" / "flecainide_fisher_before_2013.csv")
fisher_flec_after.to_csv(
    Path.cwd().parent / "data" / "flecainide_fisher_2013_and_after.csv")

In [89]:
# Compare proportions of F+B before and after 2013
data_cmp = df_cpvt_bb.copy()
data_cmp["before_2013"] = data_cmp["year"] < 2013
data_cmp = data_cmp.groupby("before_2013").agg(
    {
        "treatment_taken::Flecainide": "sum",
        "treatment_taken::Beta blocker": "sum"
    }
).rename(
    columns={
        "treatment_taken::Flecainide": "Flecainide_and_beta_blocker",
        "treatment_taken::Beta blocker": "Total"
    }
)
data_cmp["Beta_blocker_only"] = data_cmp["Total"] - data_cmp[
    "Flecainide_and_beta_blocker"]

data_cmp = data_cmp[
    ["Flecainide_and_beta_blocker", "Beta_blocker_only", "Total"]]

# rename index
data_cmp.index = ["2013 and after", "Before 2013"]
data_cmp.index.name = "Group"

data_cmp.head()

,Flecainide_and_beta_blocker,Beta_blocker_only,Total
Group,,,
2013 and after,74.0,236.0,310.0
Before 2013,37.0,76.0,113.0


In [90]:
data_cmp.to_csv(
    Path.cwd().parent / "data" / "flecainide_fisher_before_after_cmp.csv")